In [1]:
import yaml
import sqlparse
import os
import pandas as pd
os.chdir('..')
print(os.getcwd())

from omop_etl.datastore import DataStore, format_bo_sql
from omop_etl.load import Loader

# Generate cohort
omop = DataStore('config.yml')

z:\OMOP\omop_etl


In [2]:
omop = DataStore('config.yml')

def format_stage_query(dp_name):

    with open('stage_aliases.yml') as f:
        yml = yaml.safe_load(f) 
    
    aliases= yml[dp_name]
    bo_queries = omop.get_bo_query('omop')

    start_date = omop.config_param['date_range']['start_date']
    end_date = omop.config_param['date_range']['end_date']

    sql_query = format_bo_sql(bo_queries[dp_name], dp_name, schema='stage', aliases=aliases)
    patient_id = "select PATIENT_KEY from DWS_OMOP.cohort.PersonList"
    sql_query = sql_query.replace("12345678", patient_id)
    sql_query = sql_query.replace("01/01/1900 00:0:0", start_date)
    sql_query = sql_query.replace("12/31/1900 00:0:0", end_date)

    sql_query = sqlparse.format(sql_query, reindent_aligned=True, indent_with=1)

    return f"EXECUTE ('USE [DWS_PROD];\n {sql_query}')" 

In [14]:
table = 'MEASUREMENT_HeartRate'
print(format_stage_query(table))

EXECUTE ('USE [DWS_PROD];
 DROP TABLE IF EXISTS DWS_OMOP.stage.MEASUREMENT_HeartRate;SELECT DISTINCT dbo.ALL_PATIENTS.PATNT_KEY AS patient_key,
       PATNT_ENCNTR_KEY_XREF1.PATNT_ENCNTR_KEY AS patnt_encntr_key,
       Flowsheet_Enc_Dttime_VITALS.MEASR_TAKN_DATE AS vitals_date,
       Flowsheet_Enc_Dttime_VITALS.MEASR_TAKN_DT AS vitals_datetime,
       Flowsheet_VITALS_HR.MEASR_VALUE AS heartrate,
       ALL_PROVIDERS_PT_IND.PROVIDR_KEY AS attending_provider,
       ALL_PROVIDERS_VISIT.PROVIDR_KEY AS visit_provider INTO DWS_OMOP.stage.MEASUREMENT_HeartRate
  FROM dbo.ALL_PATIENTS
 RIGHT OUTER JOIN dbo.ALL_PATIENT_SNAPSHOTS ALL_PT_SNAPSHOTS_ENCOUNTER
    ON (ALL_PT_SNAPSHOTS_ENCOUNTER.PATNT_KEY=dbo.ALL_PATIENTS.PATNT_KEY)
 RIGHT OUTER JOIN dbo.PATIENT_ENCOUNTER_DTL
    ON (ALL_PT_SNAPSHOTS_ENCOUNTER.PATNT_SNAPSHT_KEY=dbo.PATIENT_ENCOUNTER_DTL.DSCHRG_PATNT_SNAPSHT_KEY AND dbo.PATIENT_ENCOUNTER_DTL.TEST_IND=''N'')
  LEFT OUTER JOIN dbo.PATNT_ENCNTR_KEY_XREF PATNT_ENCNTR_KEY_XREF1
    ON (

In [16]:
bo_queries = omop.get_bo_query('omop')
print(bo_queries.keys())

for t in bo_queries.keys():
    f = open(f'./output/sqlstring_{t.lower()}.sql', 'w')
    f.write(format_stage_query(t))
    f.close()

# aliases= yml[table]
# format_bo_sql(bo_queries[table], table, schema='stage')
# omop.execute(format_stage_query('MEASUREMENT_HeartRate'))

# aliases = {}
# for t in bo_queries.keys():
#     with omop.engine.connect() as con:
#         cols = pd.read_sql(f"select top 0 * from stage.{t}", con)
#         aliases[t] = [x.lower() for x in cols.columns]

# with open('col_aliases.yml', 'w') as f:
#     yaml.dump(aliases, f)
#     f.close()

dict_keys(['CONDITION', 'DEATH', 'DRUG_ADMIN', 'DRUG_ORDER', 'MEASUREMENT_BP', 'MEASUREMENT_HeartRate', 'MEASUREMENT_Height', 'MEASUREMENT_LAB', 'MEASUREMENT_PainScale', 'MEASUREMENT_QTCB', 'MEASUREMENT_Res_Device', 'MEASUREMENT_Res_ETCO2', 'MEASUREMENT_Res_FIO2', 'MEASUREMENT_Res_GCS', 'MEASUREMENT_Res_O2', 'MEASUREMENT_Res_PEEP', 'MEASUREMENT_Res_PIP', 'MEASUREMENT_Res_RESP', 'MEASUREMENT_Res_SPO2', 'MEASUREMENT_Res_Tidal', 'MEASUREMENT_Res_Vent', 'MEASUREMENT_Rothman', 'MEASUREMENT_SOFA', 'MEASUREMENT_Temp', 'MEASUREMENT_Weight', 'OBSERVATION_ICU', 'OBSERVATION_LDA', 'OBSERVATION_Payer', 'OBSERVATION_Smoking', 'OBSERVATION_Vent', 'OBSERVATION_Zipcode', 'PERSON', 'PROCEDURE_CPT', 'PROCEDURE_ICD', 'VISIT'])


In [3]:
bo_queries = omop.get_bo_query('omop')
sqlstring = bo_queries['DRUG_ADMIN']
parsed = sqlparse.parse(sqlstring)[0]
idx = [parsed.token_index(t) for t in parsed if t.is_keyword and t.value == 'FROM'][0]
columns = parsed.token_prev(idx)[1]

# Extract columns from SELECT clause. If duplicated columns, use alias, 
# else append abbreviated source table name.
colnames = [i.value.split('.')[-1] for i in columns]
dup_cols = set([x for x in colnames if colnames.count(x) > 1])
new_colnames = []

counter = 0
for item in columns:
    if isinstance(item, sqlparse.sql.Identifier):
        colname = item.value.split('.')[-1]
        tabname = item.value.split('.')[-2]
        shrt_tabname = '_'.join([word[:3] for word in tabname.split('_')])
        print('Identifier', item.value)
    
    elif isinstance(item, sqlparse.sql.Function):
        counter += 1
        fn_name = f'FN_{counter}'
        print('Function', item.value)
        
    else: 
        print('Something else', item.__repr__)

Identifier dbo.ALL_PATIENTS.PATNT_KEY
Something else <bound method Token.__repr__ of <Punctuation ',' at 0x1556133ACA8>>
Something else <bound method Token.__repr__ of <Whitespace ' ' at 0x1556133ABE8>>
Something else <bound method Token.__repr__ of <Whitespace ' ' at 0x1556133A4C8>>
Identifier dbo.ALL_MEDICATION_STATUSES.STNDRD_LABEL
Something else <bound method Token.__repr__ of <Punctuation ',' at 0x1556135DD68>>
Something else <bound method Token.__repr__ of <Whitespace ' ' at 0x1556135DE28>>
Something else <bound method Token.__repr__ of <Whitespace ' ' at 0x1556135DE88>>
Identifier ALL_MEDICATIONS1.LINK_RXNORM_CODE
Something else <bound method Token.__repr__ of <Punctuation ',' at 0x1556136B228>>
Something else <bound method Token.__repr__ of <Whitespace ' ' at 0x1556136B108>>
Something else <bound method Token.__repr__ of <Whitespace ' ' at 0x1556136B0A8>>
Something else <bound method Token.__repr__ of <Case 'case w...' at 0x15561369CC8>>
Something else <bound method Token.__rep

In [2]:
from omop_etl.stage import Stager
stage = Stager('config.yml')
# print(l.update_mappings('person'))
# print(l.update_mappings('visit_occurrence'))

# print(l.preload_all())

# print(l.load_table('person'))
# print(l.load_table('death'))
# print(l.load_table('condition_occurrence'))
# print(l.load_table('procedure_occurrence'))
# print(l.load_table('drug_exposure'))
# print(l.load_table('observation'))
# print(l.load_table('provider'))
# print(l.load_table('care_site'))
# print(l.load_table('location'))

In [5]:
load_conf = stage.store.config_param['load']
# stage.stage_table('measurement','res_fio2')

In [15]:
for t in load_conf.keys():
    if load_conf[t]:
        for part in load_conf[t].keys():
            stage.stage_table(t, part)
    else:
        if t not in ('provider','care_site','location'): 
            stage.stage_table(t)

no parts person
no parts death
no parts condition_occurrence
has part procedure_occurrence icd
has part procedure_occurrence cpt
has part drug_exposure order
has part drug_exposure admin
has part measurement bp
has part measurement heart_rate
has part measurement height
has part measurement lab
has part measurement pain
has part measurement qtcb
has part measurement res_dev
has part measurement res_etco2
has part measurement res_fio2
has part measurement res_gcs
has part measurement res_o2
has part measurement res_peep
has part measurement res_pip
has part measurement res_resp
has part measurement res_spo2
has part measurement res_tidal
has part measurement res_vent
has part measurement rothman
has part measurement sofa
has part measurement temp
has part measurement weight
has part observation icu
has part observation lda
has part observation vent
has part observation payer
has part observation smoking
has part observation zipcode
no parts visit
